In [1]:
import pandas as pd

df = pd.read_csv("/Users/maitrychauhan/Desktop/stockdata/stocks_old.csv")

In [2]:
df

,avg_vol_3m,change,company_name,last_price,market_cap,pe_ratio_ttm,percent_change,sector_name,symbol,volume
0,3.424M,-4.39,Berkshire Hathaway Inc.,401.5635,868481433600,9.07,-1.08%,Financial Services,BRK-B,557612
1,8.914M,-1.69,JPMorgan Chase & Co.,191.3900,549689327616,11.55,-0.88%,Financial Services,JPM,1501706
2,6.041M,-4.25,Visa Inc.,270.7700,554022469632,30.29,-1.55%,Financial Services,V,736384
3,2.41M,-5.07,Mastercard Incorporated,457.4318,426734485504,38.63,-1.10%,Financial Services,MA,378647
4,40.791M,-0.77,Bank of America Corporation,37.5500,295400964096,12.95,-2.01%,Financial Services,BAC,5972380
...,...,...,...,...,...,...,...,...,...,...
1070,"495,670",-7.20,"EPAM Systems, Inc.",237.8000,13774802944,33.68,-2.94%,Technology,EPAM,69559
1071,3.934M,-0.47,"Dynatrace, Inc.",46.2700,13695874048,70.11,-1.01%,Technology,DT,235086
1072,4.176M,-7.06,"Enphase Energy, Inc.",100.1050,13620587520,51.60,-6.59%,Technology,ENPH,1819510
1073,4.419M,-0.34,Gen Digital Inc.,20.4850,13047102464,9.27,-1.66%,Technology,GEN,182878


In [3]:
fields = [
    "Apr_1_2024",
    "Mar_1_2024",
    "Feb_1_2024",
    "Jan_1_2024",
    "Dec_1_2023",
    "Nov_1_2023",
    "Oct_1_2023",
    "Sep_1_2023",
    "Aug_1_2023",
    "Jul_1_2023",
    "Jun_1_2023",
    "May_1_2023",
    "Apr_1_2023",
    "Mar_1_2023",
    "Feb_1_2023",
    "Jan_1_2023",
    "Dec_1_2022",
    "Nov_1_2022",
    "Oct_1_2022",
    "Sep_1_2022",
    "Aug_1_2022",
    "Jul_1_2022",
    "Jun_1_2022",
    "May_1_2022",
    "Apr_1_2022",
    "Mar_1_2022",
    "Feb_1_2022",
    "Jan_1_2022",
    "Dec_1_2021",
    "Nov_1_2021",
    "Oct_1_2021",
    "Sep_1_2021",
    "Aug_1_2021",
    "Jul_1_2021",
    "Jun_1_2021",
    "May_1_2021",
    "Apr_1_2021",
    "Mar_1_2021",
    "Feb_1_2021",
    "Jan_1_2021",
    "Dec_1_2020",
    "Nov_1_2020",
    "Oct_1_2020",
    "Sep_1_2020",
    "Aug_1_2020",
    "Jul_1_2020",
    "Jun_1_2020",
    "May_1_2020",
    "Apr_1_2020",
    "Mar_1_2020",
    "Feb_1_2020",
    "Jan_1_2020",
    "Dec_1_2019",
    "Nov_1_2019",
    "Oct_1_2019",
    "Sep_1_2019",
    "Aug_1_2019",
    "Jul_1_2019",
    "Jun_1_2019",
    "May_1_2019"
]


In [4]:
for col in fields:
    df[col] = pd.NA

In [7]:

import json

with open("/Users/maitrychauhan/Desktop/stockdata/investments.history.json", "r") as f:
    datajson = json.load(f)



In [22]:
def fill_values_dates(symbol,col):
    # Retrieve the first item with a specific symbol
    result = next(filter(lambda item: item["symbol"] == symbol, datajson), None)
    if not result:
        return None
    result= result.get("history")
    getdate= next(filter(lambda item: re.sub(r'[\s,]+', '_', item['date']) == col, result), None)
    if getdate:
        return getdate.get("close")
    else:
        return None
    # return next(filter(lambda item: re.sub(r'[\s,]+', '_', item['date']) == col, result), None).get("close")

for field in fields:
    df[field] = df.apply(lambda row: fill_values_dates(row["symbol"], field), axis=1)  # Corrected
    # df[field] = df["symbol"].apply(lambda s: fill_values_dates(s, field), axis=1)

In [25]:
df.to_csv("/Users/maitrychauhan/Desktop/stockdata/stocks_appended.csv")

In [29]:
import pandas as pd
import numpy as np
import re

# Load dataset
data = pd.read_csv("/Users/maitrychauhan/Desktop/stockdata/stocks_appended_backup.csv")

# Identify date-based columns using a pattern
date_cols = [col for col in data.columns if re.match(r"\w{3}_\d{1,2}_\d{4}", col)]

# Convert date-based columns to numeric types
for col in date_cols:
    data[col] = pd.to_numeric(data[col], errors="coerce")  # Converts strings to numeric, handling errors

# Define a helper function to calculate metrics
def calculate_metrics(row, period=3):
    # Prices for recent and earlier periods
    last_price = row[date_cols[0]]
    earlier_price = row[date_cols[period - 1]]
    
    if pd.isna(last_price) or pd.isna(earlier_price):
        return None, None  # Handle missing data cases

    # Percentage change over the period
    percentage_change = (last_price - earlier_price) / earlier_price * 100

    # Volatility as standard deviation of prices over the period
    recent_prices = row[date_cols[:period]].astype(float)
    volatility = np.std(recent_prices)

    return percentage_change, volatility

# Apply metrics calculation
data['3m_change'], data['3m_volatility'] = zip(*data.apply(lambda row: calculate_metrics(row, period=3), axis=1))

# Handle missing metric values
data = data.dropna(subset=['3m_change', '3m_volatility'])  # Drop rows with missing metrics

# Classification strategy
sell_threshold = -10  # Percentage drop
high_volatility_threshold = 5  # Standard deviation

def classify_stock(row):
    if row['3m_change'] <= sell_threshold and row['3m_volatility'] >= high_volatility_threshold:
        return "Sell"
    return "Keep"

# Create a "should_sell" column
data['should_sell'] = data.apply(classify_stock, axis=1)

# Display sample output
print(data[['company_name', 'should_sell', '3m_change', '3m_volatility']])


                        company_name should_sell  3m_change  3m_volatility
0            Berkshire Hathaway Inc.        Keep  -0.151441       5.394096
1               JPMorgan Chase & Co.        Keep   1.800494       6.079037
2                          Visa Inc.        Keep  -3.647750       4.275670
3            Mastercard Incorporated        Keep  -3.793496      10.470958
4        Bank of America Corporation        Keep   9.298957       1.559922
...                              ...         ...        ...            ...
1069  Zebra Technologies Corporation        Keep  -2.808788      12.616128
1071                 Dynatrace, Inc.        Keep  -8.092836       1.717951
1072            Enphase Energy, Inc.        Sell -12.991103       6.816905
1073                Gen Digital Inc.        Keep  -4.653327       0.780043
1074      Manhattan Associates, Inc.        Keep  -9.872498      11.131328

[987 rows x 4 columns]


In [31]:
date_cols

['Apr_1_2024',
 'Mar_1_2024',
 'Feb_1_2024',
 'Jan_1_2024',
 'Dec_1_2023',
 'Nov_1_2023',
 'Oct_1_2023',
 'Sep_1_2023',
 'Aug_1_2023',
 'Jul_1_2023',
 'Jun_1_2023',
 'May_1_2023',
 'Apr_1_2023',
 'Mar_1_2023',
 'Feb_1_2023',
 'Jan_1_2023',
 'Dec_1_2022',
 'Nov_1_2022',
 'Oct_1_2022',
 'Sep_1_2022',
 'Aug_1_2022',
 'Jul_1_2022',
 'Jun_1_2022',
 'May_1_2022',
 'Apr_1_2022',
 'Mar_1_2022',
 'Feb_1_2022',
 'Jan_1_2022',
 'Dec_1_2021',
 'Nov_1_2021',
 'Oct_1_2021',
 'Sep_1_2021',
 'Aug_1_2021',
 'Jul_1_2021',
 'Jun_1_2021',
 'May_1_2021',
 'Apr_1_2021',
 'Mar_1_2021',
 'Feb_1_2021',
 'Jan_1_2021',
 'Dec_1_2020',
 'Nov_1_2020',
 'Oct_1_2020',
 'Sep_1_2020',
 'Aug_1_2020',
 'Jul_1_2020',
 'Jun_1_2020',
 'May_1_2020',
 'Apr_1_2020',
 'Mar_1_2020',
 'Feb_1_2020',
 'Jan_1_2020',
 'Dec_1_2019',
 'Nov_1_2019',
 'Oct_1_2019',
 'Sep_1_2019',
 'Aug_1_2019',
 'Jul_1_2019',
 'Jun_1_2019',
 'May_1_2019']

In [32]:
import pandas as pd
import numpy as np
import re

data = pd.read_csv("/Users/maitrychauhan/Desktop/stockdata/stocks_appended_backup.csv")

date_cols = [col for col in data.columns if re.match(r"\w{3}_\d{1,2}_\d{4}", col)]

for col in date_cols:
    data[col] = pd.to_numeric(data[col], errors="coerce")

def calculate_metrics(row, date_cols, periods=[3, 6, 12]):
    metrics = {}

    for period in periods:
        if period > len(date_cols):
            break

        last_price = row[date_cols[0]]
        earlier_price = row[date_cols[period - 1]]
        
        if pd.isna(last_price) or pd.isna(earlier_price):
            metrics[f'{period}m_change'] = None
            metrics[f'{period}m_volatility'] = None
            metrics[f'{period}m_rolling_avg'] = None
            continue

        percentage_change = (last_price - earlier_price) / earlier_price * 100

        rolling_avg = np.mean(row[date_cols[:period]])

        volatility = np.std(row[date_cols[:period]])

        # Store metrics
        metrics[f'{period}m_change'] = percentage_change
        metrics[f'{period}m_rolling_avg'] = rolling_avg
        metrics[f'{period}m_volatility'] = volatility

    return metrics

all_metrics = data.apply(lambda row: calculate_metrics(row, date_cols), axis=1)

metrics_df = pd.DataFrame(list(all_metrics))

data = pd.concat([data, metrics_df], axis=1)

data = data.dropna()

def classify_stock(row):
    if row['3m_change'] <= -10 or row['3m_volatility'] >= 5:
        return "Sell"
    elif row['3m_change'] >= 10 or row['3m_rolling_avg'] > row['6m_rolling_avg']:
        return "Buy"
    return "Hold"

data['recommendation'] = data.apply(classify_stock, axis=1)

print(data[['company_name', '3m_change', '6m_change', '12m_change', 'recommendation']])


                        company_name  3m_change  6m_change  12m_change  \
0            Berkshire Hathaway Inc.  -0.151441  13.550000   27.314065   
1               JPMorgan Chase & Co.   1.800494  21.354434   39.569671   
2                          Visa Inc.  -3.647750   6.097086   23.209519   
3            Mastercard Incorporated  -3.793496  10.371409   25.130130   
7                     Morgan Stanley   7.031613  16.070078   12.634540   
...                              ...        ...        ...         ...   
1068                    Trimble Inc.  -3.677071  27.025862   26.290979   
1069  Zebra Technologies Corporation  -2.808788  14.621487    3.450508   
1072            Enphase Energy, Inc. -12.991103   9.394179  -36.444674   
1073                Gen Digital Inc.  -4.653327  -7.201087   16.818700   
1074      Manhattan Associates, Inc.  -9.872498   2.362699   25.851615   

     recommendation  
0              Sell  
1              Sell  
2               Buy  
3              Sell  
7

In [36]:
data.to_csv("/Users/maitrychauhan/Desktop/stockdata/stocks_reccommended.csv")